### Sentencepiece : NSMC 데이터로 tokenizer 구성해보기
: https://github.com/google/sentencepiece  

내부 단어 분리를 위한 유용한 패키지 : 구글의 센텐스피스(Sentencepiece)

In [1]:
! pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/992.0 kB ? eta -:--:--
   - -------------------------------------- 30.7/992.0 kB 1.4 MB/s eta 0:00:01
   ------------------ --------------------- 460.8/992.0 kB 7.3 MB/s eta 0:00:01
   --------------------------------------- 992.0/992.0 kB 12.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import sentencepiece as spm
import csv

In [3]:
# 데이터 읽어오기 : ratings_train.txt
naver_df = pd.read_table('./data/ratings_train.txt')

In [4]:
# Null 값 제거 / 확인
print(naver_df['document'].isnull().sum())
naver_df = naver_df.dropna(how='any')
print(naver_df.isnull().values.any())

5
False


In [5]:
# 최종적으로 전처리된 텍스트를 'naver_review.txt'에 저장 (문장 \n 구분)
with open('./data/naver_review.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(naver_df['document']))
f.close()

In [ ]:
# 학습하기: 2개 파일 생성 .vocab (subword), .model
spm.SentencePieceTrainer.Train('--input=./data/naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_lengh = 9999')

In [7]:
# vocab 파일 확인해보기
vocab_list = pd.read_csv('./data/naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
4819,첸,-4816
2394,▁왜이렇게,-2391
4448,옮,-4445
4201,값,-4198
4228,첩,-4225
4046,샤,-4043
2283,▁한계,-2280
1622,와서,-1619
2136,소한,-2133
2447,▁젊은,-2444


In [8]:
# 모델 로드
sp = spm.SentencePieceProcessor()
sp.load('./data/naver.model')

True

In [9]:
# 모델을 통해 tokenizing
lines = [
    "집에 가고 싶다",
    "하지만 오늘은 즐거운 금요일",
    "흐엉엉....",
    "이럴수가.... 이렇게 간단하게 할 수 있다니"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

집에 가고 싶다
['▁집', '에', '▁가', '고', '▁싶다']
[467, 3301, 46, 3293, 763]

하지만 오늘은 즐거운 금요일
['▁하지만', '▁오늘', '은', '▁즐거', '운', '▁금', '요', '일']
[509, 960, 3310, 1566, 3405, 1715, 3318, 3379]

흐엉엉....
['▁흐', '엉', '엉', '....']
[953, 3731, 3731, 30]

이럴수가.... 이렇게 간단하게 할 수 있다니
['▁이', '럴', '수가', '....', '▁이렇게', '▁간', '단', '하게', '▁할', '▁수', '▁있다', '니']
[6, 3954, 622, 30, 249, 486, 3477, 104, 218, 60, 449, 3319]



In [10]:
# 모델로 voca size 확인하기
sp.GetPieceSize()

5000

In [11]:
# id to subword
sp.IdToPiece(4444)

'듦'

In [12]:
# subword to id
print(sp.PieceToId('▁흐')) # 언더바 아님!
print(sp.PieceToId('_흐'))

953
0


In [13]:
# ids to subwords
sp.DecodeIds([509, 960, 3310, 1566, 3405, 1715, 3318, 3379])

'하지만 오늘은 즐거운 금요일'

In [14]:
# subwords to 원형
sp.DecodePieces(['▁이', '럴', '수가', '....', '▁이렇게', '▁간', '단', '하게', '▁할', '▁수', '▁있다', '니'])

'이럴수가.... 이렇게 간단하게 할 수 있다니'

In [15]:
# encode 메소드 중 out_type 인자를 통해: subwords, ids로 변환
print(sp.Encode("집에 가고 싶다...zzzzzzzㅋㅋㅋㅋ", out_type=str))
print(sp.Encode("집에 가고 싶다...zzzzzzzㅋㅋㅋㅋ", out_type=int))

['▁집', '에', '▁가', '고', '▁싶다', '...', 'z', 'z', 'z', 'z', 'z', 'z', 'z', 'ᄏᄏᄏᄏ']
[467, 3301, 46, 3293, 763, 8, 4138, 4138, 4138, 4138, 4138, 4138, 4138, 71]
